# Problem Description
As app development is an increasingly innovative industry nowadyas, ratings in this competitve market has become a significant index to measure the success of an app. It not only has the potential to give developers a hint how to make a more popular product, but also become a determinant for users' purchase decisions. High-rating apps are more likely to be attractive to users, thus can drive business success in app-developing business. Besiedes, the vast availability of the features of existing apps published by app stores provides the possibility to predict apps' ratings.  

In this case, we mainly focus on predicting the rating of an app in Android market based on the collected features of published apps. The target variable will be the app ratings which is an average of user ratings out of 5 stars. Also, this is a problem of regression.

# Dataset Description
The dataset for Google play Store Apps is the web scraped data of about 10k Play Store apps for analysing the Android market. There are two pieces of dataset, one of them is the objective statistics of the app including size, installs .etc, another one is user reviews containing the most relevant 100 reviews of each app and attributed with some features. We suppose the first dataset will be of more contributions, so we mainly focus on the objective statistics to excavate high-rating features.  

The dataset contains 10840 entries and 11 features with each app, accompanied with the target variable. The description of each feature is given as follows:  
App: Application name  
Category: Category the app belongs to  
Rating: Overall user rating of the app out of 5 stars(as when scraped)  
Reviews: Number of user reviews for the app (as when scraped)  
Size: Size of the app (as when scraped)  
Installs: Number of user downloads/installs for the app (as when scraped)  
Type: Paid or Free  
Price: Price of the app (as when scraped)  
Content Rating: Age group the app is targeted at -- Children / Mature 21+ / Adult  
Genres: An app can belong to multiple genres (apart from its main category). e.g. a musical family game will belong to Music, Game, Family genres  
Last Updated: Date when the app was last updated on Play Store (as when scraped)  
Current Ver: Current version of the app available on Play Store (as when scraped)  
Android Ver: Min required Android version (as when scraped)

# Data Preprocessing


In [70]:
import pandas as pd
import numpy as np
df = pd.read_csv('googleplaystore.csv')

#df.isna().any()
print(df.isnull().sum(axis=0)/df.shape[0])
print(sum(df.isnull().sum(axis=1)!=0)/df.shape[0])

a = df.Type.isna()
for i in range(len(df.Type)):
    if a[i] == True:
        if df.Price[i] == "0":
            df.Type[i] = "Free"
        else:
            df.Type[i] = "Paid"

df.Type.isna().any()

df["Content Rating"].replace(to_replace=' ',value = "?", inplace=True)
df.Type.isna().any()

df.dropna(inplace = True)   #remove missing entries
df.info()



App               0.000000
Category          0.000000
Rating            0.135965
Reviews           0.000000
Size              0.000000
Installs          0.000000
Type              0.000092
Price             0.000000
Content Rating    0.000092
Genres            0.000000
Last Updated      0.000000
Current Ver       0.000738
Android Ver       0.000277
dtype: float64
0.13661101374411955
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9360 entries, 0 to 10840
Data columns (total 13 columns):
App               9360 non-null object
Category          9360 non-null object
Rating            9360 non-null float64
Reviews           9360 non-null object
Size              9360 non-null object
Installs          9360 non-null object
Type              9360 non-null object
Price             9360 non-null object
Content Rating    9360 non-null object
Genres            9360 non-null object
Last Updated      9360 non-null object
Current Ver       9360 non-null object
Android Ver       9360 non-null objec

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [49]:
import pandas as pd
import numpy as np
df = pd.read_csv('googleplaystore.csv')

Firstly, we noticed that there are some missing entries in some features and we check the fraction of missing values per row and column.

In [50]:
#df.isna().any()
print(df.isnull().sum(axis=0)/df.shape[0])
print(sum(df.isnull().sum(axis=1)!=0)/df.shape[0])

App               0.000000
Category          0.000000
Rating            0.135965
Reviews           0.000000
Size              0.000000
Installs          0.000000
Type              0.000092
Price             0.000000
Content Rating    0.000092
Genres            0.000000
Last Updated      0.000000
Current Ver       0.000738
Android Ver       0.000277
dtype: float64
0.13661101374411955


We can see that there are missing entries in Rating, Type, Content Rating, Current Ver and Android Ver. The overall perventage of missing values is about 13%, which is mostly due to the missing ratings. We try to fill in the feature of Types since it can be implied from Price, and we treat the missings in categorial feature Content Rating as another category. For the rest of features with missing entries, we will just remove the NaN entries since we have no such simple way to recover them that can remove bias. Finally we have a dataset with 9360 rows and 13 columns with no missing entries.

In [51]:
a = df.Type.isna()
for i in range(len(df.Type)):
    if a[i] == True:
        if df.Price[i] == "0":
            df.Type[i] = "Free"
        else:
            df.Type[i] = "Paid"

df.Type.isna().any()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


False

In [52]:
df["Content Rating"].replace(to_replace=' ',value = "?", inplace=True)
df.Type.isna().any()

False

In [53]:
df.dropna(inplace = True)   #remove missing entries
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9360 entries, 0 to 10840
Data columns (total 13 columns):
App               9360 non-null object
Category          9360 non-null object
Rating            9360 non-null float64
Reviews           9360 non-null object
Size              9360 non-null object
Installs          9360 non-null object
Type              9360 non-null object
Price             9360 non-null object
Content Rating    9360 non-null object
Genres            9360 non-null object
Last Updated      9360 non-null object
Current Ver       9360 non-null object
Android Ver       9360 non-null object
dtypes: float64(1), object(12)
memory usage: 1023.8+ KB


For each feature, we consider different transformations as well as some preprocessings before it can be applied with transformers.

## Catagory
We apply LabelEncoder with interger values from 0 to n_classes-1 since the categories are unordered and there is no need applying OneHotEncoder with no intersections.

In [23]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
lenc = le.fit_transform(df["Category"])
print(lenc)

[ 0  0  0 ... 11  3 18]


## Rating
We apply MinMaxScaler to Rating since it's continuous and has the range 1.0~5.0.

In [8]:
from sklearn.preprocessing import MinMaxScaler
mmscaler = MinMaxScaler()
mmscaler.fit_transform(df["Rating"].values.reshape(-1,1))

array([[0.775],
       [0.725],
       [0.925],
       ...,
       [1.   ],
       [0.875],
       [0.875]])

## Reviews
For the feature Reviews, we can see the type of the feature is object. We should first convert it into numeric, then apply StandardScaler because it varies a lot and has outlier cases.

In [9]:
df['Reviews'] = df['Reviews'].astype(int)
df.Reviews.dtypes

dtype('int64')

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit_transform(df.Reviews.values.reshape(-1,1))

array([[-0.16351077],
       [-0.16325384],
       [-0.13573493],
       ...,
       [-0.16356006],
       [-0.16352508],
       [-0.03690781]])

## Size
For the feature Size, except for "Varies with device", the rest either contain the unit "M" or "k". We should convert them into a uniform standard of numerics.

In [52]:
df.Size.value_counts()

Varies with device    1637
14M                    165
12M                    161
15M                    159
11M                    159
                      ... 
270k                     1
34k                      1
516k                     1
259k                     1
982k                     1
Name: Size, Length: 413, dtype: int64

In [53]:
df.Size=df.Size.str.replace('k','e+3')
df.Size=df.Size.str.replace('M','e+6')
df.Size

0                     19e+6
1                     14e+6
2                    8.7e+6
3                     25e+6
4                    2.8e+6
                ...        
10834                2.6e+6
10836                 53e+6
10837                3.6e+6
10839    Varies with device
10840                 19e+6
Name: Size, Length: 9360, dtype: object

In [54]:
def is_convertable(v):
    try:
        float(v)
        return True
    except ValueError:
        return False
    
temp=df.Size.apply(lambda x: is_convertable(x))
temp.head()

0    True
1    True
2    True
3    True
4    True
Name: Size, dtype: bool

In [55]:
df.Size[~temp].value_counts()

Varies with device    1637
Name: Size, dtype: int64

In [56]:
df.Size=df.Size.replace('Varies with device','')
df.Size=df.Size.replace('1,000+',1000)

In [58]:
df.Size=pd.to_numeric(df.Size)
df.Size.isna().any()
df.Size

0        19000000.0
1        14000000.0
2         8700000.0
3        25000000.0
4         2800000.0
            ...    
10834     2600000.0
10836    53000000.0
10837     3600000.0
10839           NaN
10840    19000000.0
Name: Size, Length: 9360, dtype: float64

## Installs
For the feature Installs, since it contains "," and "+" in the string, we consider to treat the 19 unique values in it as categorial and apply OrdinalEncoder to it.

## Type
For the feature Type, we can apply LaberEncoder since it's just two unordered categories

In [135]:
from sklearn.preprocessing import OrdinalEncoder
oenc = OrdinalEncoder(categories = [['Free','Paid']])
oenc.fit_transform(df.Type.values.reshape(-1,1))

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

## Price
For the feature Price, "$" should be removed and then convert to numeric. We apply MinMaxScaler to it.

In [91]:
df.Price=df.Price.apply(lambda x: x.strip('$'))

In [92]:
df.Price = pd.to_numeric(df.Price)

In [94]:
from sklearn.preprocessing import MinMaxScaler
mmscaler = MinMaxScaler()
mmscaler.fit_transform(df.Price.values.reshape(-1,1))
df.Price

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
10834    0.0
10836    0.0
10837    0.0
10839    0.0
10840    0.0
Name: Price, Length: 9360, dtype: float64

## Content Rating
We apply OrdinalEncoder to the feature since it can be arranged in the age order.

In [139]:
from sklearn.preprocessing import OrdinalEncoder
ordinalenc = OrdinalEncoder(categories = [['Teen','Everyone 10+','Mature 17+','Adults only 18+','Everyone','Unrated']])
ordinalenc.fit_transform(df["Content Rating"].values.reshape(-1,1))

array([[4.],
       [4.],
       [4.],
       ...,
       [4.],
       [2.],
       [4.]])

## Genres
For the feature Genres, we can see there are some transactions. The thing we should do first is to split the categories in each genre, and then apply OneHotEncoder.

In [1]:
a = "ej,kg

['ej', 'kg']

In [28]:
df.Genres.head()

0                 Art & Design
1    Art & Design;Pretend Play
2                 Art & Design
3                 Art & Design
4      Art & Design;Creativity
Name: Genres, dtype: object

In [22]:
df.Genres.unique()

array(['Art & Design', 'Art & Design;Pretend Play',
       'Art & Design;Creativity', 'Auto & Vehicles', 'Beauty',
       'Books & Reference', 'Business', 'Comics', 'Comics;Creativity',
       'Communication', 'Dating', 'Education;Education', 'Education',
       'Education;Creativity', 'Education;Music & Video',
       'Education;Action & Adventure', 'Education;Pretend Play',
       'Education;Brain Games', 'Entertainment',
       'Entertainment;Music & Video', 'Entertainment;Brain Games',
       'Entertainment;Creativity', 'Events', 'Finance', 'Food & Drink',
       'Health & Fitness', 'House & Home', 'Libraries & Demo',
       'Lifestyle', 'Lifestyle;Pretend Play',
       'Adventure;Action & Adventure', 'Arcade', 'Casual', 'Card',
       'Casual;Pretend Play', 'Action', 'Strategy', 'Puzzle', 'Sports',
       'Music', 'Word', 'Racing', 'Casual;Creativity',
       'Casual;Action & Adventure', 'Simulation', 'Adventure', 'Board',
       'Trivia', 'Role Playing', 'Simulation;Education',
 

In [27]:
df.Genres.infer_objects()
#help(df.infer_objects)
df.Genres.str.split(";")
df.Genres.shape

(9360,)

## Last Updated
My idea is to convert the exact data to number of days till now to get more information.

In [18]:
df["Last Updated"]

0         January 7, 2018
1        January 15, 2018
2          August 1, 2018
3            June 8, 2018
4           June 20, 2018
               ...       
10834       June 18, 2017
10836       July 25, 2017
10837        July 6, 2018
10839    January 19, 2015
10840       July 25, 2018
Name: Last Updated, Length: 9360, dtype: object

## Current Ver

In [37]:
df["Current Ver"].unique()

array(['1.0.0', '2.0.0', '1.2.4', ..., '1.5.447', '1.0.612928', '0.3.4'],
      dtype=object)

## Android Ver

In [71]:
import re
df["Current Ver"]=df["Current Ver"].apply(lambda x: 'Varies with device' if x=='Varies with device'  else  re.findall('^[0-9]\.[0-9]|[\d]|\W*',str(x))[0])

In [72]:
df["Current Ver"].unique()

array(['1.0', '2.0', '1.2', 'Varies with device', '1.1', '6.1', '2.9',
       '2.8', '3.8', '3.1', '2.2', '5.5', '4.0', '1.5', '6.0', '6.7',
       '1.6', '2.1', '1.3', '1', '1.4', '3.0', '1.7', '2.5', '2.4', '1.9',
       '0.2', '4.4', '2.3', '8.3', '3.2', '4.6', '5.4', '3.3', '9.1',
       '5.0', '9.0', '7.0', '3.9', '3.4', '9.7', '4.9', '5.2', '3.5',
       '7.3', '5.1', '4.1', '9', '0.9', '6.4', '3.7', '2.6', '4.3', '6.3',
       '4.7', '4.2', '1.8', '3', '9.8', '2', '4', '', '6', '7.5', '9.9',
       '0.1', '7.7', '7.2', '6.5', '5.6', '4.5', '5.8', '8.1', '6.6',
       '4.8', '8.7', '0.0', '3.6', '5.3', '8.2', '5.7', '8.5', '0', '8.0',
       '8.4', '6.8', '6.2', '2.7', '5.9', '7.8', '7.1', '7.9', '8.8',
       '7.4', '9.6', '8', '9.4', '5', '0.7', '0.5', '7', '7.6', '0.6',
       '8.9', '6.9', '9.3', '9.2', '8.6', '9.5', '0.8', '0.4', '0.3'],
      dtype=object)

In [73]:
a = df["Current Ver"].sort_values(ascending = False)
a

10840    Varies with device
1891     Varies with device
3838     Varies with device
3839     Varies with device
3845     Varies with device
                ...        
3673                       
8881                       
8177                       
9055                       
7736                       
Name: Current Ver, Length: 9360, dtype: object

In [75]:
df["Current Ver"] = df["Current Ver"].sort_values(ascending = False)
df["Current Ver"]

0                       1.0
1                       2.0
2                       1.2
3        Varies with device
4                       1.1
                ...        
10834                   1.0
10836                   1.4
10837                   1.0
10839    Varies with device
10840    Varies with device
Name: Current Ver, Length: 9360, dtype: object

In [83]:
df["Current Ver"].replace(to_replace="Varies with device",value = "0", inplace=True)
df["Current Ver"] = pd.to_numeric(df["Current Ver"])
a = df["Current Ver"].unique()
